replay_buffer.py

In [1]:
import threading
import numpy as np

class Buffer:
    #constuctor 
    def __init__(self, max_size=1000, num_agents=3, state_size=14, action_size=1):
        #define memory parameters
        self.size = max_size
        self.current_size = 0
        self.num_agents = num_agents
        self.buffer = dict()
        #initialize replay memory strucure for n agents
        for i in range(num_agents):
            self.buffer['state_%d' % i] = np.empty([self.size, state_size])
            self.buffer['action_%d' % i] = np.empty([self.size, action_size])
            self.buffer['reward_%d' % i] = np.empty([self.size])
            self.buffer['new_state_%d' % i] = np.empty([self.size, state_size])
        self.lock = threading.Lock()

    # store experience in replay memory
    def store_experience(self, state, action, reward, state_):
        idxs = self._get_storage_idx(inc=1)  
        for i in range(self.num_agents):
            with self.lock:
                self.buffer['state_%d' % i][idxs] = state[i]
                self.buffer['action_%d' % i][idxs] = action[i]
                self.buffer['reward_%d' % i][idxs] = reward[i]
                self.buffer['new_state_%d' % i][idxs] = state_[i]

    # sample the data from the replay buffer
    def sample(self, batch_size):
        temp_buffer = {}
        idx = np.random.randint(0, self.current_size, batch_size)
        for key in self.buffer.keys():
            temp_buffer[key] = self.buffer[key][idx]
        return temp_buffer

    #get index for new data
    def _get_storage_idx(self, inc=None):
        inc = inc or 1
        if self.current_size + inc <= self.size:
            idx = np.arange(self.current_size, self.current_size + inc)
        elif self.current_size < self.size:
            overflow = inc - (self.size - self.current_size)
            idx_a = np.arange(self.current_size, self.size)
            idx_b = np.random.randint(0, self.current_size, overflow)
            idx = np.concatenate([idx_a, idx_b])
        else:
            idx = np.random.randint(0, self.size, inc)
        self.current_size = min(self.size, self.current_size + inc)
        if inc == 1:
            idx = idx[0]
        return idx

actor_critic.py

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Actor(nn.Module):
    def __init__(self, max_action=2.1, state_size=14, action_size=1):
        super(Actor, self).__init__()
        self.max_action = max_action
        self.fc1 = nn.Linear(state_size, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 2048)
        self.fc4 = nn.Linear(2048, 4096)
        self.fc5 = nn.Linear(4096, 8192)
        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 2048)
        self.fc8 = nn.Linear(2048, 1024)
        self.fc9 = nn.Linear(1024, 512)
        self.fc10 = nn.Linear(512, 256)
        self.action_out = nn.Linear(256, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc9(x))
        x = F.relu(self.fc10(x))
        actions = self.max_action * torch.sigmoid(self.action_out(x))

        return actions


class Critic(nn.Module):
    def __init__(self, high_action=2.1, state_size=14*3, action_size=3):
        super(Critic, self).__init__()
        self.max_action = high_action
        self.fc1 = nn.Linear(state_size + action_size, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 2048)
        self.fc4 = nn.Linear(2048, 4096)
        self.fc5 = nn.Linear(4096, 8192)
        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 2048)
        self.fc8 = nn.Linear(2048, 1024)
        self.fc9 = nn.Linear(1024, 512)
        self.fc10 = nn.Linear(512, 256)
        self.action_out = nn.Linear(256, 1)

    def forward(self, state, action):
        state = torch.cat(state, dim=1)
        for i in range(len(action)):
            action[i] /= self.max_action
        action = torch.cat(action, dim=1)
        x = torch.cat([state, action], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc9(x))
        x = F.relu(self.fc10(x))
        q_value = self.action_out(x)
        return q_value

maddpg.py

In [3]:
import torch
import os


class MADDPG:
    def __init__(self, agent_id, tau=0.01, gamma=0.95, lr_actor=0.01, lr_critic=0.01):  
        self.agent_id = agent_id
        self.train_step = 0
        self.tau = tau
        self.gamma = gamma
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = 'cuda'
        # create the actor/critic
        self.actor_network = Actor().to(self.device)
        self.critic_network = Critic().to(self.device)

        # create target nets
        self.actor_target_network = Actor().to(self.device)
        self.critic_target_network = Critic().to(self.device)
        self.actor_target_network.load_state_dict(self.actor_network.state_dict())
        self.critic_target_network.load_state_dict(self.critic_network.state_dict())

        # create the optimizer
        self.actor_optim = torch.optim.Adam(self.actor_network.parameters(), lr=lr_actor)
        self.critic_optim = torch.optim.Adam(self.critic_network.parameters(), lr=lr_critic)

    # soft update
    def _soft_update_target_network(self):
        for target_param, param in zip(self.actor_target_network.parameters(), self.actor_network.parameters()):
            target_param.data.copy_((1 - self.tau) * param.data + self.tau * target_param.data)

        for target_param, param in zip(self.critic_target_network.parameters(), self.critic_network.parameters()):
            target_param.data.copy_((1 - self.tau) * param.data + self.tau * target_param.data)

    # update the network
    def train(self, transitions, other_agents, num_agents=3):
        for key in transitions.keys():
            transitions[key] = torch.tensor(transitions[key], dtype=torch.float32).to(device=self.device)
        reward = transitions['reward_%d' % self.agent_id] 
        state, action, state_ = [], [], []  
        for agent_id in range(num_agents):
            state.append(transitions['state_%d' % agent_id])
            action.append(transitions['action_%d' % agent_id])
            state_.append(transitions['new_state_%d' % agent_id])

        # calculate the target Q value function
        action_next = []
        with torch.no_grad():
            index = 0
            for agent_id in range(num_agents):
                if agent_id == self.agent_id:
                    action_next.append(self.actor_target_network(state_[agent_id]))
                else:
                    action_next.append(other_agents[index].policy.actor_target_network(state_[agent_id]))
                    index += 1
            q_next = self.critic_target_network(state, action_next).detach()
            target_q = (reward.unsqueeze(1) + self.gamma * q_next).detach()

        # the critic loss
        q_value = self.critic_network(state, action)
        critic_loss = (target_q - q_value).pow(2).mean()

        # the actor loss
        action[self.agent_id] = self.actor_network(state[self.agent_id])
        actor_loss = - self.critic_network(state, action).mean()

        #backprop losses
        self.actor_optim.zero_grad()
        actor_loss.backward()
        self.actor_optim.step()
        self.critic_optim.zero_grad()
        critic_loss.backward()
        self.critic_optim.step()

        #update target nets
        self._soft_update_target_network()
        self.train_step += 1

agent.py


In [4]:
import numpy as np
import torch
import os

class Agent:
    def __init__(self, agent_id, high_action=2.1, action_size=1):
        self.agent_id = agent_id
        self.high_action = high_action
        self.action_size = action_size
        self.policy = MADDPG(agent_id)
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = 'cuda'

    def select_action(self, state, epsilon):
        if np.random.uniform() < epsilon:
            action = np.random.randint(0, self.high_action*1000)/1000
        else:
            inputs = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
            output = self.policy.actor_network(inputs).squeeze(0)
            action = output.cpu().detach()
            action = np.clip(action, 0, self.high_action)
            action = action.item()
        return action

    def learn(self, transitions, other_agents):
        self.policy.train(transitions, other_agents)

Parameters

In [5]:
import numpy as np

f = 60
b = 0.1
a = [1 / 3, 1 / 3, 1 / 3]
D1 = [0.1, 0.2, 0.3]
H = [23.64, 6.4, 3.01]
D = np.zeros((3,))
Tsv = 2 * np.ones(3,)
Tch = 4 * np.ones(3,)
Rd = 0.05 * np.ones(3,)
Xd = [0.146, 0.8958, 1.3125]
Xdp = [0.0608, 0.1198, 0.1813]
Xq = [0.0969, 0.8645, 1.2578]
Xqp = [0.0969, 0.1969, 0.25]
Tdo = [8.96, 6.0, 5.89]
Tqo = [0.31, 0.535, 0.6]
Ka = [20, 20, 20]
Ta = [0.2, 0.2, 0.2]
Ke = [1, 1, 1]
Te = [0.314, 0.314, 0.314]
Kf = [0.063, 0.063, 0.063]
Tf = [0.35, 0.35, 0.35]


utils.py

In [6]:
def reset_env():
    Eq = [1.0587, 1.0320, 0.7198]
    Ed = [0, 0.0000, 0.7466]
    w = [376.9911, 376.9911, 376.9911]
    Efd = [1.0960, 1.0776, 2.5456]
    Rf = [0.1973, 0.1940, 0.4582]
    Vref = [1.0960, 1.0800, 1.1783]
    Vr = [1.1195, 1.1000, 3.0656]
    Tm = [1.4218, 0.0000, 1.8203]
    Psv = [1.4218, 0.0000, 1.8203]
    Pc = [1.4218, 0.0000, 1.8203]
    delta = [7.0822, -7.8707, 70.5569]
    Id = [0.4379, 0.0587, 1.6140]
    Iq = [1.3233, 0.0000, 0.7408]
    Pg = [1.4218, 0.0000, 1.8203, 0, 0, 0, 0, 0, 0]
    Qg = [0.2823, 0.0601, -0.0007, 0, 0, 0, 0, 0, 0]
    Pd = [0, 0, 0, 0, 1.3000, 0.9000, 0, 1.0000, 0]
    Qd = [0, 0, 0, 0, 0.5000, 0.3000, 0, 0.3500, 0]
    Ped = [1.4218, 0.0000, 1.8203]
    theta = [0, -7.8707, 5.1868, -4.3963, -9.3670, -6.0097, -7.8707, -7.0898, -0.6120]
    V = [1.0400, 1.0250, 1.0250, 1.0274, 0.9991, 1.0133, 1.0213, 1.0114, 1.0303]
    return Eq, Ed, w, Efd, Rf, Vref, Vr, Tm, Psv, Pc, delta, Id, Iq, Pg, Qg, Pd, Qd, Ped, theta, V

pretrain.py


In [7]:
import numpy as np
import math
import torch
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from queue import Queue


def preload_memory(replay_memory, agents, batch_size=128):
    ws = 2 * math.pi * f

    for i in range(3):
        D[i] = D1[i] * 2 * H[i] / ws

    for _ in range(1):

        Eq, Ed, w, Efd, Rf, Vref, Vr, Tm, Psv, Pc, delta, Id, Iq, Pg, Qg, Pd, Qd, Ped, theta, V = reset_env()
        print(_)
        k = 1
        o = 1
        h = 0.1
        z = np.sum(Pc)
        time = 3000
        iterations = 0
        fs = []
        queue = []
        done = False
        delta_all = [[0,0,0] for i in range(10)]
        pc_all = [[0,0,0] for i in range(10)]
        diff = [0, 0, 0]
        for t in range(time):
            if done:
                break
            ft = []
            Eq_new = []
            Ed_new = []
            delta_new = []
            w_new = []
            Efd_new = []
            Rf_new = []
            Vr_new = []
            Tm_new = []
            Psv_new = []
            state = torch.tensor(np.transpose([Id, Iq, V[0:3], theta[0:3], Eq, Ed, delta, w, Efd, Vr, Tm, Psv, diff, Pc])).float()
            for i in range(3):
                Eq_new.append(Eq[i] + h * ((-Eq[i] - (Xd[i] - Xdp[i]) * Id[i] + Efd[i]) / Tdo[i]))
                Ed_new.append(Ed[i] + h * ((-Ed[i] + (Xq[i] - Xqp[i]) * Iq[i]) / Tqo[i]))
                delta_new.append(delta[i] + h * (w[i] - ws))
                w_new.append(w[i] + h * ((Tm[i] - Ed[i] * Id[i] - Eq[i] * Iq[i] - (Xqp[i] - Xdp[i]) * Id[i] * Iq[i] - D[i] * (w[i] - ws))*ws/(2 * H[i])))
                Efd_new.append(Efd[i] + h * (-(Ke[i] + 0.0039 * np.exp(1.555 * Efd[i])) * Efd[i] + Vr[i]) / Te[i])
                Rf_new.append(Rf[i] + h * ((-Rf[i] + (Kf[i]/Tf[i])*Efd[i]) / Tf[i]))
                Vr_new.append(Vr[i] + h * (-Vr[i] + Ka[i] * Rf[i] - Ka[i] * Kf[i]/Tf[i] * Efd[i] + Ka[i] * (Vref[i] - V[i]))/Ta[i])
                Tm_new.append(Tm[i] + h * (-Tm[i] + Psv[i])/Tch[i])
                Psv_new.append(Psv[i] + h * ((-Psv[i] + Pc[i] - 1/Rd[i] * (w[i]/ws - 1))/Tsv[i]))

                if t == 0:
                    ft.append(f)
                else:
                    ft.append(f + 1 / (2 * np.pi) * delta_theta[i] / h)

            dw = w_new[2] - w[2]
            Eq = Eq_new
            Ed = Ed_new
            delta = delta_new
            w = w_new
            Efd = Efd_new
            Rf = Rf_new
            Vr = Vr_new
            Tm = Tm_new
            Psv = Psv_new
            delta_all.append(delta)
            pc_all.append(Pc)

            f_all = (ft[0] + ft[1] + ft[2]) / 3
            ACE = b * (f_all - f)
            df = dw / (2 * math.pi)

            dACE = b * df
            z = z+h*(-0.1*dACE-1/200*ACE-(np.sum(Pc)-np.sum(Pg)))

            for i in range(3):
                Pc[i] = Ped[i]+a[i]*(z-np.sum(Ped))

            actions = [Pc[0], Pc[1], Pc[2]]

            def equations(x):
                return [Ed[0] - x[1] * np.sin(np.radians(delta[0] - x[0])) + 0.0969 * x[18],
                        Eq[0] - x[1] * np.cos(np.radians(delta[0] - x[0])) - 0.0608 * x[19],

                        Ed[1] - x[3] * np.sin(np.radians(delta[1] - x[2])) + 0.1969 * x[20],
                        Eq[1] - x[3] * np.cos(np.radians(delta[1] - x[2])) - 0.1198 * x[21],

                        Ed[2] - x[5] * np.sin(np.radians(delta[2] - x[4])) + 0.2500 * x[22],
                        Eq[2] - x[5] * np.cos(np.radians(delta[2] - x[4])) - 0.1813 * x[23],

                        x[19] * x[1] * np.sin(np.radians(delta[0] - x[0])) + x[18] * x[1] * np.cos(
                            np.radians(delta[0] - x[0])) - 17.36 * x[1] * x[7] * np.sin(np.radians(x[0] - x[6])),
                        x[19] * x[1] * np.cos(np.radians(delta[0] - x[0])) - x[18] * x[1] * np.sin(
                            np.radians(delta[0] - x[0])) + 17.36 * x[1] * x[7] * np.cos(np.radians(x[0] - x[6])) - 17.36 * x[
                            1] ** 2,

                        x[21] * x[3] * np.sin(np.radians(delta[1] - x[2])) + x[20] * x[3] * np.cos(
                            np.radians(delta[1] - x[2])) - 16.00 * x[3] * x[13] * np.sin(np.radians(x[2] - x[12])),
                        x[21] * x[3] * np.cos(np.radians(delta[1] - x[2])) - x[20] * x[3] * np.sin(
                            np.radians(delta[1] - x[2])) + 16.00 * x[3] * x[13] * np.cos(np.radians(x[2] - x[12])) - 16 * x[
                            3] ** 2,

                        x[23] * x[5] * np.sin(np.radians(delta[2] - x[4])) + x[22] * x[5] * np.cos(
                            np.radians(delta[2] - x[4])) - 17.06 * x[5] * x[17] * np.sin(np.radians(x[4] - x[16])),
                        x[23] * x[5] * np.cos(np.radians(delta[2] - x[4])) - x[22] * x[5] * np.sin(
                            np.radians(delta[2] - x[4])) + 17.07 * x[5] * x[17] * np.cos(np.radians(x[4] - x[16])) - 17.07 * x[
                            5] ** 2,

                        -17.36 * x[7] * x[1] * np.sin(np.radians(x[6] - x[0])) - 3.31 * x[7] ** 2 + 1.36 * x[7] * x[9] * np.cos(
                            np.radians(x[6] - x[8])) - 11.6 * x[7] * x[9] * np.sin(np.radians(x[6] - x[8])) + 1.942 * x[7] * x[
                            11] * np.cos(np.radians(x[6] - x[10])) - 10.51 * x[7] * x[11] * np.sin(np.radians(x[6] - x[10])),
                        17.36 * x[7] * x[1] * np.cos(np.radians(x[6] - x[0])) - 39.3 * x[7] ** 2 + 1.36 * x[7] * x[9] * np.sin(
                            np.radians(x[6] - x[8])) + 11.6 * x[7] * x[9] * np.cos(np.radians(x[6] - x[8])) + 1.942 * x[7] * x[
                            11] * np.sin(np.radians(x[6] - x[10])) + 10.52 * x[7] * x[11] * np.cos(np.radians(x[6] - x[10])),

                        -Pd[4] + 1.36 * x[9] * x[7] * np.cos(np.radians(x[8] - x[6])) - 11.6 * x[9] * x[7] * np.sin(
                            np.radians(x[8] - x[6])) + 1.19 * x[9] * x[13] * np.cos(np.radians(x[8] - x[12])) - 5.97 * x[9] * x[
                            13] * np.sin(np.radians(x[8] - x[12])) - 2.55 * x[9] ** 2,
                        -Qd[4] + 1.37 * x[9] * x[7] * np.sin(np.radians(x[8] - x[6])) + 11.6 * x[9] * x[7] * np.cos(
                            np.radians(x[8] - x[6])) - 17.34 * x[9] ** 2 + 1.19 * x[9] * x[13] * np.sin(
                            np.radians(x[8] - x[12])) + 5.98 * x[9] * x[13] * np.cos(np.radians(x[8] - x[12])),

                        -Pd[5] + 1.94 * x[11] * x[7] * np.cos(np.radians(x[10] - x[6])) - 10.51 * x[11] * x[7] * np.sin(
                            np.radians(x[10] - x[6])) - 3.22 * x[11] ** 2 + 1.28 * x[11] * x[17] * np.cos(
                            np.radians(x[10] - x[16])) - 5.59 * x[11] * x[17] * np.sin(np.radians(x[10] - x[16])),
                        -Qd[5] + 1.94 * x[11] * x[7] * np.sin(np.radians(x[10] - x[6])) + 10.51 * x[11] * x[7] * np.cos(
                            np.radians(x[10] - x[6])) - 15.84 * x[11] ** 2 + 1.28 * x[11] * x[17] * np.sin(
                            np.radians(x[10] - x[16])) + 5.59 * x[11] * x[17] * np.cos(np.radians(x[10] - x[16])),

                        -16 * x[13] * x[3] * np.sin(np.radians(x[12] - x[2])) + 1.19 * x[13] * x[9] * np.cos(
                            np.radians(x[12] - x[8])) - 5.98 * x[13] * x[9] * np.sin(np.radians(x[12] - x[8])) - 2.8 * x[
                            13] ** 2 + 1.62 * x[13] * x[15] * np.cos(np.radians(x[12] - x[14])) - 13.7 * x[13] * x[15] * np.sin(
                            np.radians(x[12] - x[14])),
                        16 * x[13] * x[3] * np.cos(np.radians(x[12] - x[2])) + 1.19 * x[13] * x[9] * np.sin(
                            np.radians(x[12] - x[8])) + 5.98 * x[13] * x[9] * np.cos(np.radians(x[12] - x[8])) - 35.45 * x[
                            13] ** 2 + 1.62 * x[13] * x[15] * np.sin(np.radians(x[12] - x[14])) + 13.67 * x[13] * x[
                            15] * np.cos(np.radians(x[12] - x[14])),

                        -Pd[7] + 1.62 * x[15] * x[13] * np.cos(np.radians(x[14] - x[12])) - 13.7 * x[15] * x[13] * np.sin(
                            np.radians(x[14] - x[12])) - 2.77 * x[15] ** 2 + 1.16 * x[15] * x[17] * np.cos(
                            np.radians(x[14] - x[16])) - 9.8 * x[15] * x[17] * np.sin(np.radians(x[14] - x[16])),
                        -Qd[7] + 1.62 * x[15] * x[13] * np.sin(np.radians(x[14] - x[12])) + 13.7 * x[15] * x[13] * np.cos(
                            np.radians(x[14] - x[12])) - 23.3 * x[15] ** 2 + 1.15 * x[15] * x[17] * np.sin(
                            np.radians(x[14] - x[16])) + 9.8 * x[15] * x[17] * np.cos(np.radians(x[14] - x[16])),

                        -17.065 * x[17] * x[5] * np.sin(np.radians(x[16] - x[4])) + 1.28 * x[17] * x[11] * np.cos(
                            np.radians(x[16] - x[10])) - 5.59 * x[17] * x[11] * np.sin(np.radians(x[16] - x[10])) + 1.15 * x[
                            17] * x[15] * np.cos(np.radians(x[16] - x[14])) - 9.78 * x[17] * x[15] * np.sin(
                            np.radians(x[16] - x[14])) - 2.49 * x[17] ** 2,
                        17.065 * x[17] * x[5] * np.cos(np.radians(x[16] - x[4])) + 1.28 * x[17] * x[11] * np.sin(
                            np.radians(x[16] - x[10])) + 5.59 * x[17] * x[11] * np.cos(np.radians(x[16] - x[10])) + 1.16 * x[
                            17] * x[15] * np.sin(np.radians(x[16] - x[14])) + 9.78 * x[17] * x[15] * np.cos(
                            np.radians(x[16] - x[14])) - 32.15 * x[17] ** 2]

            x0 = np.zeros(24,)
            x0[0:18:2] = theta
            x0[1:19:2] = V
            x0[18:24:2] = Iq
            x0[19:25:2] = Id
            diff = [60 - f_all, 60 - f_all, 60 - f_all]

            x = fsolve(equations, x0)

            delta_theta = x[0:6:2] - x0[0:6:2]
            theta = x[0:18:2]
            V = x[1:19:2]

            Iq = [x[18], x[20], x[22]]

            Id = [x[19], x[21], x[23]]

            Pg[0] = 17.36 * V[0] * V[3] * np.sin(np.radians(theta[0] - theta[3]))

            Pg[1] = 16.00 * V[1] * V[6] * np.sin(np.radians(theta[1] - theta[6]))

            Pg[2] = 17.06 * V[2] * V[8] * np.sin(np.radians(theta[2] - theta[8]))

            state_ = torch.tensor(np.transpose([Id, Iq, V[0:3], theta[0:3], Eq, Ed, delta, w, Efd, Vr, Tm, Psv, diff, Pc])).float()

            if len(queue) > 100:
                queue.pop(0)
                queue.append(np.abs(diff[0]))
            else:
                queue.append(np.abs(diff[0]))

            reward = [-1, -1, -1]
            if (len(queue) > 100) and (np.average(queue) < 0.005):
                done = True
                print(iterations)
                print(actions)
                reward = [100000, 100000, 100000]
            iterations += 1
            replay_memory.store_experience(state, actions, reward, state_)
            if replay_memory.current_size > batch_size:
                memories = replay_memory.sample(batch_size)
                for agent in agents:
                    other_agents = agents.copy()
                    other_agents.remove(agent)
                    agent.learn(memories, other_agents)


main.py


In [ ]:
import numpy as np
import random
import torch
import math
import torch
from scipy.optimize import fsolve
import time

num_agents = 3
agents = []
batch_size = 128
for i in range(num_agents):
    agent = Agent(i)
    agents.append(agent)

replay_memory = Buffer()

#steady state initial conditions

ws = 2 * math.pi * f
for i in range(3):
    D[i] = D1[i] * 2 * H[i] / ws

epsilon = 1
episodes = 1002
start = time.time()

#begin training
for _ in range(episodes):
    #reset env
    Eq, Ed, w, Efd, Rf, Vref, Vr, Tm, Psv, Pc, delta, Id, Iq, Pg, Qg, Pd, Qd, Ped, theta, V = reset_env()

    #pretrain
    if _ % 25 == 0:
        print('Starting pretrain')
        preload_memory(replay_memory, agents)
    if _ > 1000:
        epsilon = 0
    k = 1
    o = 1
    h = 0.1
    fs = []
    queue = []
    done = False
    t = 0
    epsilon *= 0.99
    print(_)
    print(epsilon)
    delta_all = [[0,0,0] for i in range(10)]
    pc_all = [[0,0,0] for i in range(10)]
    diff = [0, 0, 0]
    while not done:
        ft = []
        Eq_new = []
        Ed_new = []
        delta_new = []
        w_new = []
        Efd_new = []
        Rf_new = []
        Vr_new = []
        Tm_new = []
        Psv_new = []
        state = torch.tensor(np.transpose([Id, Iq, V[0:3], theta[0:3], Eq, Ed, delta, w, Efd, Vr, Tm, Psv, diff, Pc])).float()

        #state transition
        for i in range(3):
            Eq_new.append(Eq[i] + h * ((-Eq[i] - (Xd[i] - Xdp[i]) * Id[i] + Efd[i]) / Tdo[i]))
            Ed_new.append(Ed[i] + h * ((-Ed[i] + (Xq[i] - Xqp[i]) * Iq[i]) / Tqo[i]))
            delta_new.append(delta[i] + h * (w[i] - ws))
            w_new.append(w[i] + h * ((Tm[i] - Ed[i] * Id[i] - Eq[i] * Iq[i] - (Xqp[i] - Xdp[i]) * Id[i] * Iq[i] - D[i] * (w[i] - ws))*ws/(2 * H[i])))
            Efd_new.append(Efd[i] + h * (-(Ke[i] + 0.0039 * np.exp(1.555 * Efd[i])) * Efd[i] + Vr[i]) / Te[i])
            Rf_new.append(Rf[i] + h * ((-Rf[i] + (Kf[i]/Tf[i])*Efd[i]) / Tf[i]))
            Vr_new.append(Vr[i] + h * (-Vr[i] + Ka[i] * Rf[i] - Ka[i] * Kf[i]/Tf[i] * Efd[i] + Ka[i] * (Vref[i] - V[i]))/Ta[i])
            Tm_new.append(Tm[i] + h * (-Tm[i] + Psv[i])/Tch[i])
            Psv_new.append(Psv[i] + h * ((-Psv[i] + Pc[i] - 1/Rd[i] * (w[i]/ws - 1))/Tsv[i]))

            if t == 0:
                ft.append(f)
            else:
                ft.append(f + 1 / (2 * np.pi) * delta_theta[i] / h)

        t += 1

        Eq = Eq_new
        Ed = Ed_new
        delta = delta_new
        w = w_new
        Efd = Efd_new
        Rf = Rf_new
        Vr = Vr_new
        Tm = Tm_new
        Psv = Psv_new
        delta_all.append(delta)
        pc_all.append(Pc)
        f_all = (ft[0] + ft[1] + ft[2]) / 3

        fs.append(f_all)

        def equations(x):
            return [Ed[0] - x[1] * np.sin(np.radians(delta[0] - x[0])) + 0.0969 * x[18],
                    Eq[0] - x[1] * np.cos(np.radians(delta[0] - x[0])) - 0.0608 * x[19],

                    Ed[1] - x[3] * np.sin(np.radians(delta[1] - x[2])) + 0.1969 * x[20],
                    Eq[1] - x[3] * np.cos(np.radians(delta[1] - x[2])) - 0.1198 * x[21],

                    Ed[2] - x[5] * np.sin(np.radians(delta[2] - x[4])) + 0.2500 * x[22],
                    Eq[2] - x[5] * np.cos(np.radians(delta[2] - x[4])) - 0.1813 * x[23],

                    x[19] * x[1] * np.sin(np.radians(delta[0] - x[0])) + x[18] * x[1] * np.cos(
                        np.radians(delta[0] - x[0])) - 17.36 * x[1] * x[7] * np.sin(np.radians(x[0] - x[6])),
                    x[19] * x[1] * np.cos(np.radians(delta[0] - x[0])) - x[18] * x[1] * np.sin(
                        np.radians(delta[0] - x[0])) + 17.36 * x[1] * x[7] * np.cos(np.radians(x[0] - x[6])) - 17.36 * x[
                        1] ** 2,

                    x[21] * x[3] * np.sin(np.radians(delta[1] - x[2])) + x[20] * x[3] * np.cos(
                        np.radians(delta[1] - x[2])) - 16.00 * x[3] * x[13] * np.sin(np.radians(x[2] - x[12])),
                    x[21] * x[3] * np.cos(np.radians(delta[1] - x[2])) - x[20] * x[3] * np.sin(
                        np.radians(delta[1] - x[2])) + 16.00 * x[3] * x[13] * np.cos(np.radians(x[2] - x[12])) - 16 * x[
                        3] ** 2,

                    x[23] * x[5] * np.sin(np.radians(delta[2] - x[4])) + x[22] * x[5] * np.cos(
                        np.radians(delta[2] - x[4])) - 17.06 * x[5] * x[17] * np.sin(np.radians(x[4] - x[16])),
                    x[23] * x[5] * np.cos(np.radians(delta[2] - x[4])) - x[22] * x[5] * np.sin(
                        np.radians(delta[2] - x[4])) + 17.07 * x[5] * x[17] * np.cos(np.radians(x[4] - x[16])) - 17.07 * x[
                        5] ** 2,

                    -17.36 * x[7] * x[1] * np.sin(np.radians(x[6] - x[0])) - 3.31 * x[7] ** 2 + 1.36 * x[7] * x[9] * np.cos(
                        np.radians(x[6] - x[8])) - 11.6 * x[7] * x[9] * np.sin(np.radians(x[6] - x[8])) + 1.942 * x[7] * x[
                        11] * np.cos(np.radians(x[6] - x[10])) - 10.51 * x[7] * x[11] * np.sin(np.radians(x[6] - x[10])),
                    17.36 * x[7] * x[1] * np.cos(np.radians(x[6] - x[0])) - 39.3 * x[7] ** 2 + 1.36 * x[7] * x[9] * np.sin(
                        np.radians(x[6] - x[8])) + 11.6 * x[7] * x[9] * np.cos(np.radians(x[6] - x[8])) + 1.942 * x[7] * x[
                        11] * np.sin(np.radians(x[6] - x[10])) + 10.52 * x[7] * x[11] * np.cos(np.radians(x[6] - x[10])),

                    -Pd[4] + 1.36 * x[9] * x[7] * np.cos(np.radians(x[8] - x[6])) - 11.6 * x[9] * x[7] * np.sin(
                        np.radians(x[8] - x[6])) + 1.19 * x[9] * x[13] * np.cos(np.radians(x[8] - x[12])) - 5.97 * x[9] * x[
                        13] * np.sin(np.radians(x[8] - x[12])) - 2.55 * x[9] ** 2,
                    -Qd[4] + 1.37 * x[9] * x[7] * np.sin(np.radians(x[8] - x[6])) + 11.6 * x[9] * x[7] * np.cos(
                        np.radians(x[8] - x[6])) - 17.34 * x[9] ** 2 + 1.19 * x[9] * x[13] * np.sin(
                        np.radians(x[8] - x[12])) + 5.98 * x[9] * x[13] * np.cos(np.radians(x[8] - x[12])),

                    -Pd[5] + 1.94 * x[11] * x[7] * np.cos(np.radians(x[10] - x[6])) - 10.51 * x[11] * x[7] * np.sin(
                        np.radians(x[10] - x[6])) - 3.22 * x[11] ** 2 + 1.28 * x[11] * x[17] * np.cos(
                        np.radians(x[10] - x[16])) - 5.59 * x[11] * x[17] * np.sin(np.radians(x[10] - x[16])),
                    -Qd[5] + 1.94 * x[11] * x[7] * np.sin(np.radians(x[10] - x[6])) + 10.51 * x[11] * x[7] * np.cos(
                        np.radians(x[10] - x[6])) - 15.84 * x[11] ** 2 + 1.28 * x[11] * x[17] * np.sin(
                        np.radians(x[10] - x[16])) + 5.59 * x[11] * x[17] * np.cos(np.radians(x[10] - x[16])),

                    -16 * x[13] * x[3] * np.sin(np.radians(x[12] - x[2])) + 1.19 * x[13] * x[9] * np.cos(
                        np.radians(x[12] - x[8])) - 5.98 * x[13] * x[9] * np.sin(np.radians(x[12] - x[8])) - 2.8 * x[
                        13] ** 2 + 1.62 * x[13] * x[15] * np.cos(np.radians(x[12] - x[14])) - 13.7 * x[13] * x[15] * np.sin(
                        np.radians(x[12] - x[14])),
                    16 * x[13] * x[3] * np.cos(np.radians(x[12] - x[2])) + 1.19 * x[13] * x[9] * np.sin(
                        np.radians(x[12] - x[8])) + 5.98 * x[13] * x[9] * np.cos(np.radians(x[12] - x[8])) - 35.45 * x[
                        13] ** 2 + 1.62 * x[13] * x[15] * np.sin(np.radians(x[12] - x[14])) + 13.67 * x[13] * x[
                        15] * np.cos(np.radians(x[12] - x[14])),

                    -Pd[7] + 1.62 * x[15] * x[13] * np.cos(np.radians(x[14] - x[12])) - 13.7 * x[15] * x[13] * np.sin(
                        np.radians(x[14] - x[12])) - 2.77 * x[15] ** 2 + 1.16 * x[15] * x[17] * np.cos(
                        np.radians(x[14] - x[16])) - 9.8 * x[15] * x[17] * np.sin(np.radians(x[14] - x[16])),
                    -Qd[7] + 1.62 * x[15] * x[13] * np.sin(np.radians(x[14] - x[12])) + 13.7 * x[15] * x[13] * np.cos(
                        np.radians(x[14] - x[12])) - 23.3 * x[15] ** 2 + 1.15 * x[15] * x[17] * np.sin(
                        np.radians(x[14] - x[16])) + 9.8 * x[15] * x[17] * np.cos(np.radians(x[14] - x[16])),

                    -17.065 * x[17] * x[5] * np.sin(np.radians(x[16] - x[4])) + 1.28 * x[17] * x[11] * np.cos(
                        np.radians(x[16] - x[10])) - 5.59 * x[17] * x[11] * np.sin(np.radians(x[16] - x[10])) + 1.15 * x[
                        17] * x[15] * np.cos(np.radians(x[16] - x[14])) - 9.78 * x[17] * x[15] * np.sin(
                        np.radians(x[16] - x[14])) - 2.49 * x[17] ** 2,
                    17.065 * x[17] * x[5] * np.cos(np.radians(x[16] - x[4])) + 1.28 * x[17] * x[11] * np.sin(
                        np.radians(x[16] - x[10])) + 5.59 * x[17] * x[11] * np.cos(np.radians(x[16] - x[10])) + 1.16 * x[
                        17] * x[15] * np.sin(np.radians(x[16] - x[14])) + 9.78 * x[17] * x[15] * np.cos(
                        np.radians(x[16] - x[14])) - 32.15 * x[17] ** 2]

        x0 = np.zeros(24,)
        x0[0:18:2] = theta
        x0[1:19:2] = V
        x0[18:24:2] = Iq
        x0[19:25:2] = Id
        diff = [60 - f_all, 60 - f_all, 60 - f_all]
        actions = []

        #action
        for agent in agents:
            actions.append(agent.select_action(state[agent.agent_id], epsilon=epsilon))
        for j in range(len(actions)):
            Pc[j] = actions[j]
            if Pc[j] > 2.105 or Pc[j] < -0.105:
                print('ERROR: ' + str(Pc[j]))

        #state transition
        x = fsolve(equations, x0)
        delta_theta = x[0:6:2] - x0[0:6:2]
        theta = x[0:18:2]
        V = x[1:19:2]

        Iq = [x[18], x[20], x[22]]
        Id = [x[19], x[21], x[23]]

        Pg[0] = 17.36 * V[0] * V[3] * np.sin(np.radians(theta[0] - theta[3]))
        Pg[1] = 16.00 * V[1] * V[6] * np.sin(np.radians(theta[1] - theta[6]))
        Pg[2] = 17.06 * V[2] * V[8] * np.sin(np.radians(theta[2] - theta[8]))

        state_ = torch.tensor(np.transpose([Id, Iq, V[0:3], theta[0:3], Eq, Ed, delta, w, Efd, Vr, Tm, Psv, diff, Pc])).float()

        #check if terminal state
        if not np.isclose(equations(x), np.zeros(24,)).__contains__(False) and t < 1500:
            reward = [-1, -1, -1]
        else:
            print('Failure')
            print(actions)
            reward = [-100000, -100000, -100000]
            done = True

        if len(queue) > 100:
            queue.pop(0)
            queue.append(np.abs(diff[0]))
        else:
            queue.append(np.abs(diff[0]))


        #check if terminal state
        if (len(queue) > 100) and (np.average(queue) < 0.005):
            print('Success')
            done = True
            reward = [100000, 100000, 100000]

        #train
        replay_memory.store_experience(state, actions, reward, state_)
        if replay_memory.current_size > batch_size:
            memories = replay_memory.sample(batch_size)
            for agent in agents:
                other_agents = agents.copy()
                other_agents.remove(agent)
                agent.learn(memories, other_agents)

Starting pretrain
0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


733
[1.438961795853631, 0.017161795853631052, 1.837461795853631]
103.55667614936829
0
0.99


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Failure
[0.979, 1.622, 0.322]
357.3064453601837
1
0.9801
